## Artificial and Computational Intelligence Assignment 2

## Gaming with Min-Max Algorithm - Solution template

### List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

# Things to follow

1. Use appropriate data structures to represent the graph using python libraries
2. Provide proper documentation
3. Create neat solution without error during game playing

### Coding begins here

### PEAS - Data structures and fringes that define the Agent environment goes here

We can map the PEAS (Performance, Environment, Actuators, Sensors) framework to the specific data structures, logic, and decision-making processes that define the agent’s (AI's) environment.

**1. Performance Measure (P):**
Goal: Maximize the chances of the AI winning the game while preventing the opponent (human player) from winning.
Static Evaluation Function: This is the performance measure for the AI, scoring the current board state to evaluate how favorable it is for the AI.
Positive Scores: AI is close to winning (e.g., forming a row of 4 pieces).
Negative Scores: Opponent is close to winning, and AI needs to block.
The AI will aim to maximize this score in each turn by evaluating future board states through the min-max algorithm with alpha-beta pruning.
Winning Condition: The AI maximizes performance by getting four consecutive pieces vertically, horizontally, or diagonally, while minimizing the opponent’s chances of doing the same.

**2. Environment (E):**
The environment consists of:
*Game Board (board):*
A 6-row by 7-column grid represented as a 2D NumPy array (board).
The board is updated by dropping pieces (for either the human or AI) into one of the columns.
Empty cells are represented by 0, player pieces by 1, and AI pieces by 2.

*Game Rules:*
Players take turns placing their pieces into columns.
A win occurs when four consecutive pieces are aligned vertically, horizontally, or diagonally.

*Game States:*
A terminal state occurs if either player wins or the board is full with no valid moves left (draw).
Valid Locations: The AI must select a column with at least one open spot.

**3. Actuators (A):**
Actuators represent the actions that the agent can take to interact with the environment. In this case, the AI’s actuators are the actions that modify the board:

Drop Piece (place_piece):
The AI places its piece into one of the valid columns.
The function modifies the board by updating the state of the chosen row and column.

**4. Sensors (S):**
The sensors define how the agent perceives the environment. The AI perceives the board through data structures and functions that provide information about the current game state:

Board Representation (board): The AI senses the state of the game via the 2D NumPy array representing the board.
Valid Locations (get_valid_locations): This function tells the AI which columns are available for placing a piece.
Next Open Row (get_next_open_row): The AI uses this function to determine the row where a piece can be dropped in a valid column.
Winning Move (winning_move): The AI checks whether it (or the human player) has already won by looking for four consecutive pieces in any direction.

**Data Structures:**
Board (Environment): The central data structure is the 2D NumPy array representing the game board:
This array is updated as moves are made, and its state is evaluated using various functions like score_position, winning_move, and get_valid_locations.
Window (Evaluation): A "window" is a subarray of four consecutive cells that the AI evaluates during its decision-making process.These windows are used to score positions for potential winning or blocking opportunities in the game.
Valid Locations: A list of available columns for the next move, which is recalculated after each turn:
valid_locations = get_valid_locations(board)

**Fringe:**
In the min-max search, the fringe is represented by the potential game states that are explored as the AI evaluates its moves:

 Min-max Depth: The depth-limited tree search explores game states up to a predefined depth (DEPTH = 3).
 
 Alpha-Beta Pruning: This optimizes the minmax algorithm by eliminating branches that do not need to be explored, reducing the fringe size and speeding up decision-making.


In [29]:
import numpy as np
#from math import inf as infinity

ROWS = 6
COLUMNS = 7
PLAYER = 0
AI = 1
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2
WINDOW_LENGTH = 4
DEPTH = 3

def create_board():
    board = np.zeros((ROWS, COLUMNS))
    return board

def place_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROWS-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROWS):
        if board[r][col] == 0:
            return r

def print_board(board):
    print(np.flip(board, 0))

def winning_move(board, piece):
    # Check horizontal locations
    for c in range(COLUMNS-3):
        for r in range(ROWS):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations
    for c in range(COLUMNS):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diagonals
    for c in range(COLUMNS-3):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diagonals
    for c in range(COLUMNS-3):
        for r in range(3, ROWS):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

### Choice and implementation of the Static Evaluation Function.

In [30]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0

    ## Score center column
    center_array = [int(i) for i in list(board[:, COLUMNS//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    ## Score Horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMNS-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    ## Score Vertical
    for c in range(COLUMNS):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROWS-3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    ## Score positive sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLUMNS-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    ## Score negative sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLUMNS-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0

### Implementation of the Min-Max algorithm and alpha-beta pruning  

In [31]:
def minmax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else: # Game is over, no more valid moves
                return (None, 0)
        else: # Depth is zero
            return (None, score_position(board, AI_PIECE))
    if maximizingPlayer:
        value = -np.inf
        column = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            place_piece(b_copy, row, col, AI_PIECE)
            new_score = minmax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else: # Minimizing player
        value = np.inf
        column = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            place_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minmax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMNS):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def pick_best_move(board, piece):
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = np.random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board, col)
        temp_board = board.copy()
        place_piece(temp_board, row, col, piece)
        score = score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col

    return best_col

### Main Block

In [32]:
def main():
    board = create_board()
    print_board(board)
    game_over = False
    turn = PLAYER

    while not game_over:

        if turn == PLAYER:            
            try:
                col = int(input("Player 1- Select column no. indexed as 0-6 from left to right:"))
                if col < 0 or col >= COLUMNS:
                    raise ValueError("Column is out of range. Select any column from 0-6")
            except ValueError:
                print("Invalid input. Please enter an integer between 0 and 6.")
                continue

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                place_piece(board, row, col, PLAYER_PIECE)

                if winning_move(board, PLAYER_PIECE):
                    print("PLAYER 1 WINS!!")
                    game_over = True

        else: # AI Turn
            print("Player 2 Selection")
            col, minmax_score = minmax(board, DEPTH, -np.inf, np.inf, True)

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                place_piece(board, row, col, AI_PIECE)

                if winning_move(board, AI_PIECE):
                    print("PLAYER 2 WINS!!")
                    game_over = True

        print_board(board)
        print("========================================================")

        turn += 1
        turn = turn % 2

        if len(get_valid_locations(board)) == 0:
            print("It's a Draw!")
            game_over = True

In [ ]:
#Start the game

if __name__ == "__main__":
    main()

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Player 1- Select column no. indexed as 0-6 from left to right:7
Invalid input. Please enter an integer between 0 and 6.
Player 1- Select column no. indexed as 0-6 from left to right:1
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]
Player 2 Selection
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 2. 0. 0. 0.]]
